In [1]:
import re
import os
import requests
import hashlib
import json
import time
import random
import string
import pandas as pd
from tqdm import tqdm
import codecs
from glob import glob
from bs4 import BeautifulSoup
from scrapy.selector import Selector
from urllib.parse import unquote
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
# 定义随机的useragent
from fake_useragent import UserAgent
ua = UserAgent()


In [3]:
params = {
    'ie': 'UTF-8',
    'wd': 'Python'
}

In [4]:
ipdata = pd.read_excel('/Users/my/Desktop/代理ip的地址2000.xlsx')
ipdata.fillna("",inplace = True)

proxieslist = []

for i in ipdata.index.values:
    line = ipdata.loc[i,['HTTP']].to_dict()
    proxieslist.append(line)
    
randomproxy = random.choice(proxieslist)

In [5]:
headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }

In [6]:
def changeparams():
    headers_pc = {
    'User-Agent': ua.random,
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7'
    }
    randomproxy = random.choice(proxieslist)
     # 访问百度服务器，获取cookie
    try:
        res = requests.get('https://zhidao.baidu.com', headers=headers_pc, proxies= randomproxy,)
        # 将cookieJar数据转化为字典，
        cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    except Exception as e:
        time.sleep(5)
        res = requests.get('https://zhidao.baidu.com', headers=headers_pc, proxies= randomproxy,)
        # 将cookieJar数据转化为字典，
        cookie_dict = requests.utils.dict_from_cookiejar(res.cookies)
    return headers_pc, randomproxy, cookie_dict

In [7]:
def get_response(url,headers_pc, randomproxy, cookie_dict):
    try:
        r = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, cookies=cookie_dict, timeout=10)
        r.encoding = 'utf-8'
        return r.text
    except Exception as e:
        time.sleep(5)
        print(e)

    return None

def get_img(url,headers_pc, randomproxy, cookie_dict):
    try:
        response = requests.get(url, headers=headers_pc, params=params, proxies=randomproxy, 
                                cookies=cookie_dict, timeout=10, 
                                stream=True)
        return response
    except Exception as e:
        time.sleep(5)
        print("图片下载页访问异常：", e)

    return None

def get_md5(file_name):
    hash = hashlib.md5()
    hash.update(file_name.encode('utf-8'))
    return hash.hexdigest()

In [8]:
def img_download(headers_pc, randomproxy, cookie_dict, item):
    qa_results = None
    try:
#         title = item['title']
        url = item['url']
        newtitle = item['title'].strip().replace('?', '').replace('/', '、').replace('\\', ' ')
        
#         pbar.set_description(newtitle)
        
        if os.path.exists(f"/Users/my/Desktop/百度知道_药品问答/html/{newtitle}.html"):
            pbar.update(1)
            return None 
#         time.sleep(random.randint(0, 5))
        response = get_response(url, headers_pc, randomproxy, cookie_dict)
        
        selector = Selector(text=response)
        # titles = selector.xpath("//article/div/h1/span/text()").extract()
#         if len(titles) < 1:
#             print("访问异常")
#             time.sleep(random.randint(0, 3))
            # return None
        # title = titles[0]
        
        with open(f"/Users/my/Desktop/百度知道_药品问答/html/{newtitle}.html", 'w+', encoding='utf-8') as f:
            f.write(response)

        
        img_urls = selector.xpath("//article//div[@class='q-img-wp']//ul/li/@data-src").extract()
        if len(img_urls) == 0:
            return None
        
        img_paths = []
        for img_url in img_urls:
            img_response = get_img(img_url,headers_pc, randomproxy, cookie_dict)
            if img_response is None:
                continue
            
            img_id = get_md5(img_url)
            img_path = f'/Users/my/Desktop/百度知道_药品问答/img7.26/{img_id}.png'
            if not os.path.exists(img_path):
                with open(img_path, "wb") as f:
                    for chunk in img_response.iter_content(chunk_size=1024):
                        if chunk:
                            f.write(chunk)
                            f.flush()
            print("存储成功")
            img_paths.append(img_path)    
        
        
        if len(img_paths) == 0:
            return None
        
    except Exception as e:
        print(e)
    pbar.update(1)
    return None

In [9]:
def task_monitor(task_list, mt_pool, qa_results):
    count = 0
    while len(task_list) > 0:
        tmp_task_list = []
        for future in task_list:
            if future.done():
                result = future.result()
                # print(len(result))
                if result is None:
                    continue
                count += 1
                qa_results.append(result)
#                 #if count % 10 == 0:
#                 with open('百度知道_药品问答/qa_results_supplyment.json', 'w', encoding='utf-8') as f:
#                     f.write(json.dumps(qa_results, indent=2, ensure_ascii=False)) 
            else:
                tmp_task_list.append(future)
        
        task_list = tmp_task_list

In [ ]:
if __name__ == "__main__":
    with open('/Users/my/Desktop/百度知道_药品问答/检索词7.21.json', 'r', encoding='utf-8') as f:
        results = json.load(f)
    
    qa_results = []
    all_craw_tasks = []
    count = 0
    headers_pc, randomproxy,cookie_dict = changeparams()
    
    pbar = tqdm(total=len(results))
    
    with ThreadPoolExecutor(max_workers=10) as mt_pool:
        for item in results:
#             count += 1
#             if count % 20:
#                 headers_pc, randomproxy, cookie_dict = changeparams()
            tmp_task = mt_pool.submit(img_download, headers_pc, randomproxy, cookie_dict, item)
            all_craw_tasks.append(tmp_task) 
        # 监测下载任务进展
        task_monitor(all_craw_tasks, mt_pool, qa_results)
        
#         with open('百度知道_药品问答/qa_results_supplyment.json', 'w', encoding='utf-8') as f:
#             f.write(json.dumps(qa_results, indent=2, ensure_ascii=False))


 17%|██████▏                              | 13555/81849 [01:30<26:17, 43.30it/s]

存储成功


 17%|██████▏                              | 13556/81849 [01:31<27:22, 41.57it/s]

存储成功


 17%|█████▋                            | 13572/81849 [04:07<20:48:10,  1.10s/it]

...

 21%|██████▋                         | 17254/81849 [2:21:00<29:26:32,  1.64s/it]

HTTPConnectionPool(host='zhidao.baidu.com', port=80): Max retries exceeded with url: /question/1945764107868847988.html?fr=iks&word=%22%E7%94%B2%E6%B3%BC%E5%B0%BC%E9%BE%99%E7%89%87%22&ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15c790400>: Failed to establish a new connection: [Errno 51] Network is unreachable'))
Selector needs either text or root argument
HTTPConnectionPool(host='zhidao.baidu.com', port=80): Max retries exceeded with url: /question/1837992444821371180.html?fr=iks&word=%22%E7%94%B2%E6%B3%BC%E5%B0%BC%E9%BE%99%E7%89%87%22&ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15f0704c0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Selector needs either text or root argument
HTTPConnectionPool(host='zhidao.baidu.com', port=80): Max retries exceeded with url: /question/1545225000659694147.html?fr=iks&word=%22%E7%94%B2%E6%B3%BC%E

 21%|██████▋                         | 17257/81849 [2:21:00<17:40:06,  1.02it/s]

HTTPConnectionPool(host='zhidao.baidu.com', port=80): Max retries exceeded with url: /question/303606007.html?fr=iks&word=%22%E7%94%B2%E6%B3%BC%E5%B0%BC%E9%BE%99%E7%89%87%22&ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15f0768b0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Selector needs either text or root argument
HTTPConnectionPool(host='zhidao.baidu.com', port=80): Max retries exceeded with url: /question/1436279234472151499.html?fr=iks&word=%22%E7%94%B2%E6%B3%BC%E5%B0%BC%E9%BE%99%E7%89%87%22&ie=UTF-8&wd=Python (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x15f05c070>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Selector needs either text or root argument


 21%|██████▉                          | 17288/81849 [2:21:08<3:33:14,  5.05it/s]